## A simple model for demand and supply of geolocalized services in a city

### Load libraries and classes

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt

import os
import sys
nb_dir = os.path.dirname(os.getcwd()) ## TODO: find way to put this into some global settings
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg, istat_kpi, city_settings

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model
from src.models.core import ServiceUnit, ServiceValues, ServiceEvaluator, \
    MappedPositionsFrame, DemandFrame, KPICalculator
from src.models.factories import UnitFactory
from src.models.process_tools import GridMaker, ValuesPlotter, JSONWriter

In [ ]:
selected_city = 'Milano'

### Load service data: locations, scales, other info for city

In [ ]:
loaders=UnitFactory.make_loaders_for_city(city_settings.get_city_config(selected_city))
loaders

In [ ]:
library_units = loaders['Biblioteche'].load(mean_radius=0.4)


In [ ]:
# Initialise with a default lengthscale of 0.5 km
school_units = loaders['Scuole'].load(mean_radius=0.5)

# Initialise with a default lengthscale of 0.6 km
pharmacy_units = loaders['Farmacie'].load(mean_radius=0.5)

# Initialise with a default lengthscale of 0.3 km
## awful hack to quickly clean tpl file #FIXME
loaders['Fermate TPL']._raw_data = loaders['Fermate TPL']._raw_data[
    loaders['Fermate TPL']._raw_data['route_type'].isin([0,1,3])].reset_index()

tpl_units = loaders['Fermate TPL'].load(mean_radius=0.3)

# Initialise with a default lengthscale of 0.5 km
library_units = loaders['Biblioteche'].load(mean_radius=0.4)

### Demand import from Censimento Popolazione e Abitazioni (CPA) 2011

In [ ]:
demand_data = DemandFrame.create_from_istat_cpa(selected_city)

b_demo = False
if b_demo:
    # demo mode, use a portion of the data
    demand_test = DemandFrame(demand_data.sample(50, random_state=80).copy(), False)
    test_units = school_units[::50] + pharmacy_units[::30] + tpl_units[::30] 
    calculator = KPICalculator(demand_test, test_units, selected_city)
else:
    calculator = KPICalculator(
        demand_data, school_units + pharmacy_units + tpl_units + library_units, 
        selected_city)

### Evaluate the services offer at the demand points and average over neighbourhood

In [ ]:
# print current value of kernel cutoff
print('Ignoring interactions below %s \n' % common_cfg.kernel_value_cutoff)

# compute and plot demand/supply interaction for localized services 
calculator.evaluate_services_at_demand(b_evaluate_attendance=True, clip_level=1.4)  # this might take a while.

calculator.compute_kpi_for_localized_services()

for _,loader in loaders.items():
    pass
    loader.save_units_with_attendance_to_geojson(calculator.evaluator.units)

In [ ]:
ptt = ValuesPlotter(calculator.service_values)
ptt.plot_service_levels(ServiceType.Library)


In [ ]:
for service_type, units in calculator.evaluator.units_tree.items():
    values = [u.attendance for u in units]
    print(min(values), max(values))
    sns.distplot(values, 80)
    plt.title(service_type)
    plt.show()

### Append Istat KPI and export averaged values by Neighbourhood-Service-AgeGroup to JSON

In [ ]:
# compute istat kpi as well
calculator.compute_kpi_for_istat_values()

# write KPI to json outputb
json_maker = JSONWriter(calculator)
#jsonMaker.write_all_files_to_default_path()

In [ ]:
b_save_pickle = False
import pickle
import time
filename = './pickles/'+ selected_city + '_%i.pickle' % int(time.time())
if b_save_pickle:
    with open(filename, 'wb') as f:
        pickle.dump(calculator, f)